In [19]:
import sys
sys.path.insert(0, "/Users/aimachine/PyImage/utils")
import numpy as np

import importlib
import matplotlib.pyplot as plt
import os
import glob
import skimage
from skimage import measure
from skimage import filters
from Normalize import normalizeFloat, normalizeMinMax, Path, save_tiff_imagej_compatible
from skimage.filters import threshold_otsu
from tifffile import imread
import scipy
from scipy import ndimage
from skimage.measure import compare_ssim as ssim
from bokeh.models import Label
from bokeh.io import output_file, show
from bokeh.plotting import figure
from bokeh.io import export_png, output_notebook
from scipy import stats
from skimage.measure import compare_ssim as ssim

In [20]:
Lowbasedir = '/Users/aimachine/Downloads/GrantApplication/StructureSimilarity/SmallLow/'
GTbasedir = '/Users/aimachine/Downloads/GrantApplication/StructureSimilarity/SmallGT/'
Restoredbasedir = '/Users/aimachine/Downloads/GrantApplication/StructureSimilarity/SmallRestored/'

In [21]:
LowPath = os.path.join(Lowbasedir, '*.tif')
GTPath = os.path.join(GTbasedir, '*.tif')
RestoredPath = os.path.join(Restoredbasedir, '*.tif')
AllLow = []
AllGT = []
AllRestored = []
Names = []
filesLow = glob.glob(LowPath)
filesGT = glob.glob(GTPath)
filesRestored = glob.glob(RestoredPath)


for fname in filesLow:
      x = imread(fname)
      """Normalize"""
      x = normalizeFloat(x)
      AllLow.append(x) 
      print(fname)
      for filenameGT in filesGT:
       
        if os.path.basename(filenameGT) ==  os.path.basename(fname):
                  x = imread(filenameGT)
                  filesGT.remove(filenameGT)
                  """Normalize"""
                  x = normalizeFloat(x)
                  AllGT.append(x)
      for filenameRestored in filesRestored:
  
        if os.path.basename(filenameRestored) == 'BorialisS1S2FlorisMidNoiseModel_' + os.path.basename(fname):
                  x = imread(filenameRestored)
                  filesRestored.remove(filenameRestored)
                  """Normalize"""
                  x = normalizeFloat(x)
                  AllRestored.append(x)  
     
      Names.append(fname)


AllLow.sort
AllGT.sort
AllRestored.sort
Names.sort
print(len(AllLow), len(AllRestored), len(AllGT))

/Users/aimachine/Downloads/GrantApplication/StructureSimilarity/SmallLow/40x_bin1_05_5min_s1-4Cg_s5-6esg_1_w2CSU-488_s1_t28.tif
/Users/aimachine/Downloads/GrantApplication/StructureSimilarity/SmallLow/40x_bin1_05_5min_s1-4Cg_s5-6esg_1_w2CSU-488_s1_t29.tif
/Users/aimachine/Downloads/GrantApplication/StructureSimilarity/SmallLow/40x_bin1_05_5min_s1-4Cg_s5-6esg_1_w2CSU-488_s1_t39.tif
/Users/aimachine/Downloads/GrantApplication/StructureSimilarity/SmallLow/40x_bin1_05_5min_s1-4Cg_s5-6esg_1_w2CSU-488_s1_t38.tif
/Users/aimachine/Downloads/GrantApplication/StructureSimilarity/SmallLow/40x_bin1_05_5min_s1-4Cg_s5-6esg_1_w2CSU-488_s1_t35.tif
/Users/aimachine/Downloads/GrantApplication/StructureSimilarity/SmallLow/40x_bin1_05_5min_s1-4Cg_s5-6esg_1_w2CSU-488_s1_t34.tif
/Users/aimachine/Downloads/GrantApplication/StructureSimilarity/SmallLow/40x_bin1_05_5min_s1-4Cg_s5-6esg_1_w2CSU-488_s1_t36.tif
/Users/aimachine/Downloads/GrantApplication/StructureSimilarity/SmallLow/40x_bin1_05_5min_s1-4Cg_s5-6esg

In [22]:
Z = AllLow[0].shape[0]

In [23]:
def mse(x, y):
    return np.linalg.norm(x - y)

In [24]:

GTMSE = []
RestoredMSE = []

LowSNR = []
GTSNR = []
RestoredSNR = []
min = 0;
max = 1;


for i in range(0, len(AllLow)):
 Low = AllLow[i]
 GT = AllGT[i]
 Restored = AllRestored[i]

 for j in range(Z):
      smallLowimg = Low[j,:,:]
      smallGTimg = GT[j,:,:]
      smallRestoredimg = Restored[j,:,:]


     

    
      smallGTMSE = mse(smallGTimg, smallLowimg)
      smallLowssim = ssim(smallGTimg, smallLowimg, data_range=smallLowimg.max() - smallLowimg.min())
     
      
      GTMSE.append(smallGTMSE)
      LowSNR.append(smallLowssim)
     
        
      smallRestoredMSE = mse(smallGTimg, smallRestoredimg)
      smallRestoredssim = ssim(smallGTimg, smallRestoredimg, data_range=smallRestoredimg.max() - smallRestoredimg.min())
      RestoredMSE.append(smallRestoredMSE)
      RestoredSNR.append(smallRestoredssim)  
      

/Users/aimachine/anaconda3/envs/tensorflowpy3pt5/lib/python3.5/site-packages/skimage/util/arraycrop.py:177: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  cropped = ar[slices]


In [49]:
hist, edges = np.histogram(np.abs(GTMSE), density=False, bins=5)
p = figure(title="Max Count MeanSquared Error (Low/GT) = " + str('%.3f'%edges[np.argmax(hist)]), y_axis_label='Counts',x_axis_label='MSE')
listedges = edges.tolist()
listhist = hist.tolist()

p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
     fill_color="#043564", line_color="#643304")
for  i in range(0,len(listhist)):
 if listhist[i] > 0:
  mytext = Label(x=listedges[i], y=listhist[i], text=str('%.2f'%listedges[i]))
  #p.add_layout(mytext)
output_notebook()    
show(p)

histSec, edgesSec = np.histogram(np.abs(RestoredMSE), density=False, bins=5)
q = figure(title="Max Count MeanSquared Error(Restored/GT) = " + str('%.3f'%edges[np.argmax(histSec)]), y_axis_label='Counts',x_axis_label='MSE')
listedgesSec = edgesSec.tolist()
listhistSec = histSec.tolist()

q.quad(top=histSec, bottom=0, left=edgesSec[:-1], right=edgesSec[1:],
    fill_color="#036564", line_color="#033649")
for  i in range(0,len(listhistSec)):
 if listhistSec[i] > 0:
  mytext = Label(x=listedgesSec[i], y=listhistSec[i], text=str('%.2f'%listedgesSec[i]))
  #p.add_layout(mytext)
output_notebook()    
show(q)


q.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
        fill_color="#043564", line_color="#643304")

show(q)


Loading BokehJS ...

Loading BokehJS ...

In [50]:
hist, edges = np.histogram(np.abs(LowSNR), density=False, bins=15)
p = figure(title="Max Count SIM(Low) = " + str('%.3f'%edges[np.argmax(hist)]), y_axis_label='Counts',x_axis_label='SIM')
listedges = edges.tolist()
listhist = hist.tolist()

p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
     fill_color="#043564", line_color="#643304")

for  i in range(0,len(listhist)):
 if listhist[i] > 0:
  mytext = Label(x=listedges[i], y=listhist[i], text=str('%.2f'%listedges[i]))
 
  #p.add_layout(mytext)

output_notebook()    
show(p)




histSec, edgesSec = np.histogram(np.abs(RestoredSNR), density=False, bins=15)
q = figure(title="Max Count SIM(Restored) = " + str('%.3f'%edges[np.argmax(histSec)]), y_axis_label='Counts',x_axis_label='SIM')
listedgesSec = edgesSec.tolist()
listhistSec = histSec.tolist()

q.quad(top=histSec, bottom=0, left=edgesSec[:-1], right=edgesSec[1:],
    fill_color="#036564", line_color="#033649")

for  i in range(0,len(listhistSec)):
 if listhistSec[i] > 0:
  mytext = Label(x=listedgesSec[i], y=listhistSec[i], text=str('%.2f'%listedgesSec[i]))
  
  #p.add_layout(mytext)

output_notebook()    
show(q)

q.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
    fill_color="#043564", line_color="#643304")

show(q)




Loading BokehJS ...

259.63336137962807


Loading BokehJS ...

242.84198015984688
